In [1]:
# Для работы с данными
import pandas as pd
import numpy as np

#Для работы с моделями
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.model_selection import cross_val_score

#Для визуализации
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_data = pd.read_csv('train.csv')

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
# Находим не категориальные признаки
cat_feat_train = list(train_data.dtypes[(train_data.dtypes == object)].index)
cat_feat_test = list(train_data.dtypes[(train_data.dtypes == object)].index)

train_data[cat_feat_train] = train_data[cat_feat_train].fillna('nan')

cat_nunique = train_data[cat_feat_train].nunique()
cat_feat = list(cat_nunique[cat_nunique < 30].index)

# Не категориальные признаки
num_feat = [f for f in train_data if f not in (cat_feat + ['ID', 'SalePrice'])]

In [24]:
# Создаем дамми-переменные для категориальных признаков
dummy_train = pd.get_dummies(train_data[cat_feat], columns=cat_feat)

dummy_train_cols = list(set(dummy_train))

dummy_train = dummy_train[dummy_train_cols]

train = pd.concat([train_data[num_feat].fillna(-999), dummy_train], axis=1)


In [9]:
X = train.loc[:, train.columns]
y = train_data['SalePrice']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [12]:
model_forest = RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_leaf=10, max_features=0.5, n_jobs=-1)


In [13]:
score = cross_val_score(model_forest, X_train, y_train, cv=10)
# Оценка модели с помощью кросс-валидации на 10-fold
score.mean()

0.8249501645407886

In [14]:
# Зафитим модель и посчитаем предсказания для тестовой выборки
model_forest.fit(X,y)
forest_predictions = model_forest.predict(X_test)

In [15]:
# Оценка качества модели исходя из метрики, обозначенной на Kaggle - Mean Squared Log Error
mean_squared_log_error(y_test, forest_predictions)

0.0246225444632483

In [17]:
#Модели для получения мета-признаков
# from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

#Инструмент для разбиения на фолды
from sklearn.model_selection import StratifiedKFold

In [18]:
# Три модели для стекинга
stack_models = [ # Случайный лес
                 DecisionTreeRegressor(),
                 # Логистическая регрессия
                 SVR(kernel='linear', gamma='scale'),
                 # Линейная регрессия
                 LinearRegression()
                ]

In [20]:
# Функция для получения мета-признаков
def get_meta_features(clf, X_train, y_train, stack_cv):
    meta_train = np.zeros_like(y_train, dtype=float)
    meta_test = np.zeros_like(y_test, dtype=float)
    scoring = []
    
    for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train, y_train)):
        clf.fit(X_train.iloc[train_ind], y_train.iloc[train_ind])
        meta_train[test_ind] = clf.predict(X_train.iloc[test_ind])
        # meta_test += clf.predict(X_test)
        scoring += [clf.score(X_train.iloc[test_ind], y_train.iloc[test_ind])]
        
    print('Scoring: {0}'.format(scoring))
    
    return meta_train #, meta_test / stack_cv.n_splits

In [21]:
# Разбиваем на фолды
stack_cv = StratifiedKFold(n_splits=4, random_state=123)

C:\FORPIT2\FFF\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [22]:
# Получаем метапризнаки для мета-модели
meta_train = []
meta_test = []
col_names = []

# Создание мета-признаков
for i in stack_models:
    print("Обучение и создание признаков для модели {0}".format(i.__class__.__name__))
    meta_train_new = get_meta_features(i, X_train, y_train, stack_cv) #, meta_test_new 
    meta_train.append(meta_train_new)
    #meta_test.append(meta_test_new)
    col_names.append(i.__class__.__name__)

Обучение и создание признаков для модели DecisionTreeRegressor


C:\FORPIT2\FFF\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


Scoring: [0.4332917045632807, 0.7277552883503096, 0.5160263538913914, 0.6271757202891131]
Обучение и создание признаков для модели SVR


C:\FORPIT2\FFF\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


Scoring: [0.7208383594960248, 0.542436354335138, 0.7868688606614034, 0.7974946480847067]
Обучение и создание признаков для модели LinearRegression


C:\FORPIT2\FFF\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


Scoring: [-2.4876451836874933, 0.7166184560600586, 0.8501301991025526, 0.8867038453460074]


In [23]:
# Собираем полученные мета-признаки, для обучения мета-модели
X_meta_train = pd.DataFrame({col_names[i]: meta_train[i] for i in range(len(col_names))})